In [8]:
import snowflake.connector
import configparser
# Loading credentials from config.ini
config = configparser.ConfigParser()
config.read("config.ini")
SNOWFLAKE_CONFIG = {
   "user": config["snowflake"]["user"],
   "password": config["snowflake"]["password"],
   "account": config["snowflake"]["account"],
   "warehouse": config["snowflake"]["warehouse"],
   "database": config["snowflake"]["database"],
   "schema": config["snowflake"]["schema"],
}
def fetch_latest_email_alert():
   conn = snowflake.connector.connect(**SNOWFLAKE_CONFIG)
   cursor = conn.cursor()
   query = """
       SELECT subject, INCIDENT_ID, impact_level, affected_service, timestamp, description
       FROM cleaned_alerts
       ORDER BY timestamp DESC
       LIMIT 1;
   """
   cursor.execute(query)
   latest_alert = cursor.fetchone()
   column_names = [desc[0] for desc in cursor.description]
   cursor.close()
   conn.close()
   if latest_alert:
       return dict(zip(column_names, latest_alert))
   return None
# Fetch the latest alert
latest_alert = fetch_latest_email_alert()
print("Latest Email Alert:", latest_alert)

Latest Email Alert: {'SUBJECT': 'Urgent: Critical System Failure Detected in Production', 'INCIDENT_ID': 'INC-20250319-001', 'IMPACT_LEVEL': None, 'AFFECTED_SERVICE': 'Payment Processing Gateway', 'TIMESTAMP': datetime.datetime(2025, 3, 27, 11, 31, 11), 'DESCRIPTION': 'A critical system failure has been detected in the Production Environment\r\nat 03:25 AM UTC. Immediate attention is required to prevent further service\r\ndisruptions.'}


In [9]:
import requests
import json

# Hugging Face API details
API_URL = "https://api-inference.huggingface.co/models/mistralai/Mixtral-8x7B-Instruct-v0.1"
API_KEY = config["huggingface"]["api_key"]
headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}

def generate_solution(alert_data):
    if not alert_data:
        return "No new alerts found."

    # Formatting alert data for AI model
    formatted_alert = f"""
    Incident ID: {alert_data['INCIDENT_ID']}
    Subject: {alert_data['SUBJECT']}
    Impact Level: {alert_data['IMPACT_LEVEL']}
    Affected Service: {alert_data['AFFECTED_SERVICE']}
    Timestamp: {alert_data['TIMESTAMP']}
    Description: {alert_data['DESCRIPTION']}
    """
    data = {
        "inputs": f"Analyze this email alert and provide a solution:\n\n{formatted_alert}"
    }

    try:
        response = requests.post(API_URL, headers=headers, json=data)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        result = response.json()
        
        if 'error' in result:
            return f"Error from API: {result['error']}"
        
        # Extracting the generated solution text
        generated_solution = result[0]['generated_text'] if isinstance(result, list) and result else "Solution not found in response."
        
        # Printing prompt and solution separately
        print("\nPrompt:")
        print(data["inputs"])
        print("\nSolution:")
        print(generated_solution)
        
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except json.JSONDecodeError:
        print("Failed to decode JSON response from API.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


# Generating and printing the solution
generate_solution(latest_alert)



Prompt:
Analyze this email alert and provide a solution:


    Incident ID: INC-20250319-001
    Subject: Urgent: Critical System Failure Detected in Production
    Impact Level: None
    Affected Service: Payment Processing Gateway
    Timestamp: 2025-03-27 11:31:11
    Description: A critical system failure has been detected in the Production Environment
at 03:25 AM UTC. Immediate attention is required to prevent further service
disruptions.
    

Solution:
Analyze this email alert and provide a solution:


    Incident ID: INC-20250319-001
    Subject: Urgent: Critical System Failure Detected in Production
    Impact Level: None
    Affected Service: Payment Processing Gateway
    Timestamp: 2025-03-27 11:31:11
    Description: A critical system failure has been detected in the Production Environment
at 03:25 AM UTC. Immediate attention is required to prevent further service
disruptions.
    
    Root Cause: The root cause is still being investigated and additional analysis
is curr